In [1]:
from itertools import combinations
# Define the points in 3D space
points = [0, 1, 2, 3, 4, 5, 6]

# Define the initial edges between the points
edges =[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 6), (2, 3), (2, 5), (3, 6), (4, 5), (4, 6), (5, 6)]
# Convert edges to a set for fast lookup
edge_set = set(edges)

# Create a helper function to check if an edge exists between two points
def has_edge(p1, p2):
    return (p1, p2) in edge_set or (p2, p1) in edge_set

# Get all pairs of points that are non-edges
all_pairs = list(combinations(points, 2))
non_edges = [pair for pair in all_pairs if not has_edge(pair[0], pair[1])]

# Function to find tetrahedra for a given non-edge
def find_tetrahedra(non_edge):
    p1, p2 = non_edge
    other_points = [p for p in points if p != p1 and p != p2]

    tetrahedra = []

    # Check all combinations of 2 points to form tetrahedra
    for triple in combinations(other_points, 2):
        p3, p4 = triple

        # Check if the four points form a valid tetrahedron (all edges present)
        if has_edge(p1, p3) and has_edge(p1, p4) and has_edge(p2, p3) and has_edge(p2, p4) and has_edge(p3, p4):
            # Store the edges of this tetrahedron (excluding the non-edge)
            edges_in_tetrahedron = [
                (p1, p3), (p1, p4), (p2, p3), (p2, p4), (p3, p4)
            ]
            tetrahedra.append(edges_in_tetrahedron)

    return tetrahedra

# Now process all non-edges
for non_edge in non_edges:
    # Find tetrahedra for the current non-edge
    tetrahedra = find_tetrahedra(non_edge)

    # Number of tetrahedra the non-edge is part of
    num_tetrahedra = len(tetrahedra)

    # Initialize the sum of shared edges
    shared_edge_sum = 0

    # Compare each pair of tetrahedra to count shared edges
    for t1, t2 in combinations(tetrahedra, 2):
        shared_edges = set(t1).intersection(t2)
        shared_edge_sum += len(shared_edges)

    # Output the results for this non-edge
    print(f"Non-edge {non_edge} is part of {num_tetrahedra} tetrahedra.")
    print(f"Total number of shared edges between its tetrahedra: {shared_edge_sum}\n")


Non-edge (0, 6) is part of 3 tetrahedra.
Total number of shared edges between its tetrahedra: 4

Non-edge (1, 5) is part of 3 tetrahedra.
Total number of shared edges between its tetrahedra: 4

Non-edge (2, 4) is part of 2 tetrahedra.
Total number of shared edges between its tetrahedra: 2

Non-edge (2, 6) is part of 1 tetrahedra.
Total number of shared edges between its tetrahedra: 0

Non-edge (3, 4) is part of 2 tetrahedra.
Total number of shared edges between its tetrahedra: 2

Non-edge (3, 5) is part of 1 tetrahedra.
Total number of shared edges between its tetrahedra: 0



In [3]:
from sage.graphs.graph_generators import graphs
import csv
from itertools import combinations

# Generate non-isomorphic graphs with 9 vertices and 21 edges
graph_generator = graphs.nauty_geng("9 21")
graphs_list = list(graph_generator)

# Define the points (vertices) and possible edges
points = list(range(9))
possible_edges = list(combinations(points, 2))

# Create a dictionary mapping each edge to its index in the vector
edge_to_index = {edge: idx for idx, edge in enumerate(possible_edges)}

# Helper function to check if an edge exists between two points
def has_edge(p1, p2, edge_set):
    return (p1, p2) in edge_set or (p2, p1) in edge_set

# Function to find tetrahedra for a given non-edge
def find_tetrahedra(non_edge, edge_set):
    p1, p2 = non_edge
    other_points = [p for p in points if p != p1 and p != p2]

    tetrahedra = []

    # Check all combinations of 2 points to form tetrahedra
    for triple in combinations(other_points, 2):
        p3, p4 = triple
        # Check if these four points form a valid tetrahedron (all edges present)
        if (has_edge(p1, p3, edge_set) and has_edge(p1, p4, edge_set) and
            has_edge(p2, p3, edge_set) and has_edge(p2, p4, edge_set) and
            has_edge(p3, p4, edge_set)):
            # Store the edges of this tetrahedron (excluding the non-edge)
            edges_in_tetrahedron = [(p1, p3), (p1, p4), (p2, p3), (p2, p4), (p3, p4)]
            tetrahedra.append(edges_in_tetrahedron)

    return tetrahedra

# List to store vectorized graphs
vectorized_graphs_tetrahedra = []
vectorized_graphs_shared_edges = []

# Process each graph in the list
for G in graphs_list:
    # Get the set of edges for the current graph
    edge_set = set(G.edges(labels=False))

    # Initialize two vectors for the current graph (one for each dataset)
    vector_tetrahedra = [0] * len(possible_edges)
    vector_shared_edges = [0] * len(possible_edges)

    # Identify non-edges in the graph
    non_edges = [pair for pair in possible_edges if not has_edge(pair[0], pair[1], edge_set)]

    # Process each non-edge
    for non_edge in non_edges:
        # Find tetrahedra containing the non-edge
        tetrahedra = find_tetrahedra(non_edge, edge_set)
        
        # Count number of tetrahedra and shared edges
        num_tetrahedra = len(tetrahedra)
        shared_edge_sum = 0
        
        # Count shared edges among the tetrahedra
        for t1, t2 in combinations(tetrahedra, 2):
            shared_edges = set(t1).intersection(t2)
            shared_edge_sum += len(shared_edges)

        # Store results in the vectors
        index = edge_to_index[non_edge]
        vector_tetrahedra[index] = num_tetrahedra
        vector_shared_edges[index] = shared_edge_sum

    # Append the vectors for this graph to the respective datasets
    vectorized_graphs_tetrahedra.append(vector_tetrahedra)
    vectorized_graphs_shared_edges.append(vector_shared_edges)

# Write the datasets to CSV files
with open("vectorized_graphs3.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([f"Edge_{i+1}" for i in range(len(possible_edges))])
    writer.writerows(vectorized_graphs_tetrahedra)

with open("vectorized_graphs_4.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([f"Edge_{i+1}" for i in range(len(possible_edges))])
    writer.writerows(vectorized_graphs_shared_edges)
